In [ ]:
# --- Célula 1: Instalação de Bibliotecas ---
%pip install -q google-genai google-adk

In [ ]:
# --- Célula 2: Importação de Bibliotecas ---
import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import asyncio
from IPython.display import display, Markdown
import warnings

# Ignorar avisos desnecessários
warnings.filterwarnings("ignore")


In [ ]:
# --- Célula 3: Configuração da API Key ---
# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY_1"] = userdata.get('GOOGLE_API_KEY_1')


In [ ]:
# --- Célula 4: Inicialização do Cliente e Serviço de Sessão ---
# Inicializa o cliente da SDK do Gemini
client = genai.Client()

# Cria um serviço de sessão em memória para gerenciar interações do agente
session_service = InMemorySessionService()


In [ ]:
# --- Célula 5: Função `call_agent` e Lógica de Retry ---
import asyncio  # Importar asyncio para trabalhar com funções assíncronas

# Função auxiliar para enviar uma mensagem para um agente via Runner
async def call_agent(agent: str, message_text: str) -> str:
    """
    Envia uma mensagem para um agente e retorna a resposta.
    Implementa lógica de retry em caso de falhas (ex.: cota excedida).
    """
    max_retries = 5  # Número máximo de tentativas
    initial_delay = 1  # Atraso inicial em segundos

    for attempt in range(max_retries):
        try:
            # Simulação: aqui você deve implementar a lógica real para chamar um agente
            content = f"Agente {agent} respondeu a: {message_text}"
            return content

        except Exception as e:  # Captura erros genéricos
            print(f"Erro inesperado na tentativa {attempt + 1}/{max_retries}: {e}")
            await asyncio.sleep(initial_delay)  # Atraso antes de tentar novamente

    raise Exception(f"Falha após {max_retries} tentativas de chamar o agente {agent}.")  # Exceção após várias falhas

# Definindo funções dos agentes
async def agente_buscador(topico: str, data_de_hoje: str) -> str:
    return await call_agent('Buscador', f"Busque informações sobre {topico} em {data_de_hoje}.")

async def agente_planejador(topico: str, lancamentos_buscados: str) -> str:
    return await call_agent('Planejador', f"Planeje um post sobre {topico} baseado nos resultados: {lancamentos_buscados}.")

async def agente_redator(topico: str, plano_de_post: str) -> str:
    return await call_agent('Redator', f"Crie um rascunho do post sobre {topico}: {plano_de_post}.")

async def agente_revisor(topico: str, rascunho_de_post: str) -> str:
    return await call_agent('Revisor', f"Revise o seguinte post sobre {topico}: {rascunho_de_post}.")


In [ ]:
# --- Célula 6: Listagem dos Modelos Disponíveis ---
# Lista todos os modelos disponíveis atualmente
for model in client.models.list():
    print(model.name)




models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview

In [ ]:
# --- Célula 7: Declaração do Cliente e do Modelo ---
from google import genai

# Inicializa o cliente do GeneAI
client = genai.Client()

# Declaração do modelo a ser utilizado
MODEL_ID = "gemini-2.0-flash"  # Você pode escolher o modelo que deseja usar

In [ ]:
# --- Célula 8: Exemplo de Pergunta ao Modelo (sem busca) ---
# Pergunta ao Gemini uma informação mais recente sem a busca do Google
try:
    response = client.models.generate_content(
        model=MODEL_ID,  # Utiliza a variável MODEL_ID
        contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    )
    # Processar e exibir a resposta
    print(response)

except genai.errors.ClientError as e:
    print(f"Ocorreu um erro: {e}")

# Exibe a resposta em um formato markdown para melhor visualização
display(Markdown(f"Resposta:\n {response.text}"))  # Corrigido de 'resposta' para 'response'


candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='Para saber quando será a próxima Imersão IA com Google Gemini da Alura, a melhor forma é verificar diretamente no site da Alura:\n\n*   **Site da Alura:** Procure na seção de cursos ou eventos relacionados a Inteligência Artificial ou Google. A Alura costuma divulgar as datas das próximas imersões e cursos por lá.\n\nAlém disso, você pode seguir a Alura nas redes sociais, pois eles costumam anunciar as novidades por lá também.')], role='model'), citation_metadata=None, finish_message=None, token_count=None, finish_reason=<FinishReason.STOP: 'STOP'>, url_context_metadata=None, avg_logprobs=-0.26014406656481553, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)] create_time=None response_id=None model_version='gemini-2.0-flash' prompt_feedback

Resposta:
 Para saber quando será a próxima Imersão IA com Google Gemini da Alura, a melhor forma é verificar diretamente no site da Alura:

*   **Site da Alura:** Procure na seção de cursos ou eventos relacionados a Inteligência Artificial ou Google. A Alura costuma divulgar as datas das próximas imersões e cursos por lá.

Além disso, você pode seguir a Alura nas redes sociais, pois eles costumam anunciar as novidades por lá também.

In [ ]:
# --- Célula 9: Exemplo de Pergunta ao Modelo (com busca) ---
# Pergunta ao Gemini utilizando a busca do Google como contexto
try:
    response = client.models.generate_content(
        model=MODEL_ID,  # Agora utiliza a variável MODEL_ID para referenciar o modelo
        contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
        config={"tools": [{"google_search": {}}]}  # Configura a busca do Google como contexto
    )
    # Exibe a resposta em um formato markdown para melhor visualização
    display(Markdown(f"Resposta:\n {response.text}"))

except genai.errors.ClientError as e:
    print(f"Ocorreu um erro: {e}")  # Tratamento de erros caso a chamada à API falhe

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As inscrições estão abertas até o dia 11 de maio de 2025.

In [ ]:
# --- Célula 11: Definição dos Agentes ---

# Agente 1: Buscador de Notícias
def agente_buscador(topico, data_de_hoje):
    """
    Agente responsável por buscar notícias sobre um tópico específico.
    """
    buscador = Agent(
        name="agente_buscador",
        model=MODEL_ID,  # Utiliza a variável MODEL_ID para referência ao modelo
        instruction="""
        Você é um assistente de pesquisa. Sua tarefa é usar a ferramenta de busca do Google
        para recuperar as últimas notícias sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]  # As ferramentas utilizadas pela agente
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)  # Chama o agente
    return lancamentos

# Agente 2: Planejador de posts
def agente_planejador(topico, lancamentos_buscados):
    """
    Agente responsável por planejar posts baseando-se nos lançamentos encontrados.
    """
    planejador = Agent(
        name="agente_planejador",
        model=MODEL_ID,  # Utiliza a variável MODEL_ID para referência ao modelo
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes, crie um plano sobre os pontos mais relevantes para um post.
        Escolha o tema mais relevante e retorne suas sugestões.
        """,
        description="Agente que planeja posts",
        tools=[google_search]  # As ferramentas utilizadas pela agente
    )

    entrada_do_agente_planejador = f"Tópico: {topico}\nLançamentos buscados: {lancamentos_buscados}"
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)  # Chama o agente
    return plano_do_post

# Agente 3: Redator do Post
def agente_redator(topico, plano_de_post):
    """
    Agente responsável por redigir o post a partir do plano dado.
    """
    redator = Agent(
        name="agente_redator",
        model=MODEL_ID,  # Utiliza a variável MODEL_ID para referência ao modelo
        instruction="""
        Você é um Redator Criativo responsável por criar posts para redes sociais.
        Use o tema e o plano fornecidos para redigir um rascunho engajador.
        """,
        description="Agente redator de posts engajadores"
    )

    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    rascunho = call_agent(redator, entrada_do_agente_redator)  # Chama o agente
    return rascunho

# Agente 4: Revisor de Qualidade
def agente_revisor(topico, rascunho_gerado):
    """
    Agente responsável por revisar o post antes da publicação.
    """
    revisor = Agent(
        name="agente_revisor",
        model=MODEL_ID,  # Utiliza a variável MODEL_ID para referência ao modelo
        instruction="""
        Você é um Editor e Revisor de Conteúdo. Revise o rascunho do post, verificando clareza e tomificação.
        """,
        description="Agente revisor de post para redes sociais."
    )

    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)  # Chama o agente
    return texto_revisado

In [80]:
# --- Célula 12: Executar o Sistema com Entrada do Usuário ---
from datetime import date  # Certifique-se de que a biblioteca date está importada
from IPython.display import display, Markdown  # Para exibir o Markdown corretamente
import asyncio  # Importando asyncio para usar com await

# Define a data atual
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post: ")

# Lógica do sistema de agentes

if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")
    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📑 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📑 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📑 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📑 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")


   # Executando a função assíncrona
   asyncio.run(executar_agentes())

display(Markdown(md))

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post: agentes de IA
Maravilha! Vamos então criar o post sobre novidades em agentes de IA

--- 📑 Resultado do Agente 1 (Buscador) ---



Erro: Conteúdo inválido recebido: <coroutine object call_agent at 0x7d903b069930>

--------------------------------------------------------------

--- 📑 Resultado do Agente 2 (Planejador) ---



Erro: Conteúdo inválido recebido: <coroutine object call_agent at 0x7d903b069a60>

--------------------------------------------------------------

--- 📑 Resultado do Agente 3 (Redator) ---



...

Agente name='agente_redator' description='Agente redator de posts engajadores' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um Redator Criativo responsável por criar posts para redes sociais. \n        Use o tema e o plano fornecidos para redigir um rascunho engajador.\n        ' global_instruction='' tools=[] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: agentes de IA
Plano de post: Agente name='agente_planejador' description='Agente que planeja posts' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de\n        lançamentos mais recentes, crie um plano sobre os pontos mais relevantes para um post.\n        Escolha o tema mais relevante e retorne suas sugestões.\n        ' global_instruction='' tools=[<google.adk.tools.google_search_tool.GoogleSearchTool object at 0x7d903d650710>] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: agentes de IA
Lançamentos buscados: Agente name='agente_buscador' description='Agente que busca informações no Google' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um assistente de pesquisa. Sua tarefa é usar a ferramenta de busca do Google\n        para recuperar as últimas notícias sobre o tópico abaixo.\n        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias.\n        ' global_instruction='' tools=[<google.adk.tools.google_search_tool.GoogleSearchTool object at 0x7d903d650710>] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: agentes de IA
Data de hoje: 01/06/2025

--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



Agente name='agente_revisor' description='Agente revisor de post para redes sociais.' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um Editor e Revisor de Conteúdo. Revise o rascunho do post, verificando clareza e tomificação.\n        ' global_instruction='' tools=[] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: agentes de IA
Rascunho: Agente name='agente_redator' description='Agente redator de posts engajadores' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um Redator Criativo responsável por criar posts para redes sociais. \n        Use o tema e o plano fornecidos para redigir um rascunho engajador.\n        ' global_instruction='' tools=[] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: agentes de IA
Plano de post: Agente name='agente_planejador' description='Agente que planeja posts' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de\n        lançamentos mais recentes, crie um plano sobre os pontos mais relevantes para um post.\n        Escolha o tema mais relevante e retorne suas sugestões.\n        ' global_instruction='' tools=[<google.adk.tools.google_search_tool.GoogleSearchTool object at 0x7d903d650710>] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: agentes de IA
Lançamentos buscados: Agente name='agente_buscador' description='Agente que busca informações no Google' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um assistente de pesquisa. Sua tarefa é usar a ferramenta de busca do Google\n        para recuperar as últimas notícias sobre o tópico abaixo.\n        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias.\n        ' global_instruction='' tools=[<google.adk.tools.google_search_tool.GoogleSearchTool object at 0x7d903d650710>] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: agentes de IA
Data de hoje: 01/06/2025

--------------------------------------------------------------



### 🧩 Fluxo concluído — *01/06/2025*

<details open>
<summary>📰 Agente 1 · Buscador</summary>

Agente name='agente_buscador' description='Agente que busca informações no Google' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um assistente de pesquisa. Sua tarefa é usar a ferramenta de busca do Google\n        para recuperar as últimas notícias sobre o tópico abaixo.\n        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias.\n        ' global_instruction='' tools=[<google.adk.tools.google_search_tool.GoogleSearchTool object at 0x7d903d650710>] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: Agentes de IA
Data de hoje: 01/06/2025
</details>

<details>
<summary>🗂️ Agente 2 · Planejador</summary>

Agente name='agente_planejador' description='Agente que planeja posts' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de\n        lançamentos mais recentes, crie um plano sobre os pontos mais relevantes para um post.\n        Escolha o tema mais relevante e retorne suas sugestões.\n        ' global_instruction='' tools=[<google.adk.tools.google_search_tool.GoogleSearchTool object at 0x7d903d650710>] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: Agentes de IA
Lançamentos buscados: Agente name='agente_buscador' description='Agente que busca informações no Google' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um assistente de pesquisa. Sua tarefa é usar a ferramenta de busca do Google\n        para recuperar as últimas notícias sobre o tópico abaixo.\n        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias.\n        ' global_instruction='' tools=[<google.adk.tools.google_search_tool.GoogleSearchTool object at 0x7d903d650710>] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: Agentes de IA
Data de hoje: 01/06/2025
</details>

<details>
<summary>✍️ Agente 3 · Redator</summary>

Agente name='agente_redator' description='Agente redator de posts engajadores' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um Redator Criativo responsável por criar posts para redes sociais. \n        Use o tema e o plano fornecidos para redigir um rascunho engajador.\n        ' global_instruction='' tools=[] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: Agentes de IA
Plano de post: Agente name='agente_planejador' description='Agente que planeja posts' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de\n        lançamentos mais recentes, crie um plano sobre os pontos mais relevantes para um post.\n        Escolha o tema mais relevante e retorne suas sugestões.\n        ' global_instruction='' tools=[<google.adk.tools.google_search_tool.GoogleSearchTool object at 0x7d903d650710>] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: Agentes de IA
Lançamentos buscados: Agente name='agente_buscador' description='Agente que busca informações no Google' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um assistente de pesquisa. Sua tarefa é usar a ferramenta de busca do Google\n        para recuperar as últimas notícias sobre o tópico abaixo.\n        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias.\n        ' global_instruction='' tools=[<google.adk.tools.google_search_tool.GoogleSearchTool object at 0x7d903d650710>] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: Agentes de IA
Data de hoje: 01/06/2025
</details>

<details>
<summary>🪄 Agente 4 · Revisor (*versão final do post*)</summary>

Agente name='agente_revisor' description='Agente revisor de post para redes sociais.' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um Editor e Revisor de Conteúdo. Revise o rascunho do post, verificando clareza e tomificação.\n        ' global_instruction='' tools=[] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: Agentes de IA
Rascunho: Agente name='agente_redator' description='Agente redator de posts engajadores' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um Redator Criativo responsável por criar posts para redes sociais. \n        Use o tema e o plano fornecidos para redigir um rascunho engajador.\n        ' global_instruction='' tools=[] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: Agentes de IA
Plano de post: Agente name='agente_planejador' description='Agente que planeja posts' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de\n        lançamentos mais recentes, crie um plano sobre os pontos mais relevantes para um post.\n        Escolha o tema mais relevante e retorne suas sugestões.\n        ' global_instruction='' tools=[<google.adk.tools.google_search_tool.GoogleSearchTool object at 0x7d903d650710>] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: Agentes de IA
Lançamentos buscados: Agente name='agente_buscador' description='Agente que busca informações no Google' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='\n        Você é um assistente de pesquisa. Sua tarefa é usar a ferramenta de busca do Google\n        para recuperar as últimas notícias sobre o tópico abaixo.\n        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias.\n        ' global_instruction='' tools=[<google.adk.tools.google_search_tool.GoogleSearchTool object at 0x7d903d650710>] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None examples=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None respondeu a: Tópico: Agentes de IA
Data de hoje: 01/06/2025
</details>
